# A : Setup

##A.1 Authentication and Checking User Details.

In [ ]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Getting the email account that was used to login to this session
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
print(gcloud_tokeninfo['email'])

##A.2 Setup the Project Parameters.

In [ ]:
# Change the following if you are not in the rsa-dev-10 project
# These parameters are generated outside of this notebook
# Project: rsa-mla-prod

# Project: rsa-dev-10
project_id = 'gcp-rsa-project-project'
bucket_id = 'gcp-rsa-project-bucket'

ip_addr_name = 'gcp-rsa-prod'

# These are parameters you setup for this deployment
region = 'us-central1'
zone = 'us-central1-f'


In [ ]:
import os
os.environ['DEVSHELL_PROJECT_ID'] = project_id
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

!gcloud config set project {project_id} -q

In [ ]:
!gcloud config set compute/zone {zone}
!gcloud config set compute/region {region}

### Install Kubectl

In [ ]:
!curl -LO https://storage.googleapis.com/kubernetes-release/release/`curl -s \
 https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl

In [ ]:
!chmod +x kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version --client

##A.3 Enabling APIs

In [ ]:
!gcloud services enable container.googleapis.com --async
!gcloud services enable containerregistry.googleapis.com --async
!gcloud services enable compute.googleapis.com --async
!gcloud services enable storage-api.googleapis.com --async
!gcloud services enable storage-component.googleapis.com --async                        
!gcloud services enable servicemanagement.googleapis.com --async
!gcloud services enable iam.googleapis.com --async
!gcloud services enable cloudbuild.googleapis.com --async

# B : Git Repository Pull

In [ ]:
%cd ~
!git clone https://github.com/skramana1973/gcp-ms.git

In [ ]:
%cd ~/gcp-ms/backend
%ls -l

In [ ]:
!./setup.sh

If ./setup give error than do below steps for npm install else skip these npm steps 
%cd ~/gcp-ms/backend
!npm install -g npm 
%cd ./monolith
!npm install
%cd ../microservices
!npm install
%cd ../react-app
!npm install
!npm run build 

In [ ]:
!ls -lrt

In [ ]:
!gcloud container clusters create fancy-cluster --num-nodes 3

In [ ]:
!gcloud compute instances list

Get Credentials

In [ ]:
!gcloud config set container/use_application_default_credentials true
!gcloud container clusters get-credentials fancy-cluster
!kubectl cluster-info

# C: Build Container Image

In [ ]:
%cd ~/gcp-ms/backend/microservices/src/orders
%pwd

In [ ]:
%cd ~/gcp-ms/backend/microservices/src/orders
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0 .

In [ ]:
!kubectl create deployment orders --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0

In [ ]:
!kubectl expose deployment orders --type=LoadBalancer --port 80 --target-port 8081

In [ ]:
!kubectl get service orders

##Build Product Container Image and Deploy It

In [ ]:
%cd ~/gcp-ms/backend/microservices/src/products
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0 .

In [ ]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

In [ ]:
!kubectl expose deployment products --type=LoadBalancer --port 80 --target-port 8082

## Build Frontend Container and Deploy It

In [ ]:
!kubectl get service

### Copy the IP address of order and product LoadBalancer and paste it below

In [ ]:
order_ip='34.72.89.148'
product_ip='34.72.212.80'

In [ ]:
"""
%cd ~/gcp-ms/backend/react-app
!sed -i 's+/service/orders+http://{order_ip}/api/orders+g' ./.env.monolith
!sed -i 's+/service/products+http://{product}/api/products+g' ./.env.monolith
!cp ./.env.monolith ./.env
!cat ./.env
"""

In [ ]:
%cd ~/gcp-ms/backend/react-app
!cp ./.env.monolith ./.env
!sed -i 's+/service/orders+http://{order_ip}/api/orders+g' ./.env
!sed -i 's+/service/products+http://{product_ip}/api/products+g' ./.env
!cat ./.env

In [ ]:
!npm run build

In [ ]:
%cd ~/gcp-ms/backend/microservices/src/frontend
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0 .

In [ ]:
!kubectl create deployment frontend --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0

In [ ]:
!kubectl expose deployment frontend --type=LoadBalancer --port 80 --target-port 8080

## How can I Get the External-IP address?

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get pods

In [ ]:
!kubectl get service

In [ ]:
!kubectl get -o jsonpath="{.status.loadBalancer.ingress[0].ip}" --namespace=default services frontend

## Resilency Test

In [ ]:
!kubectl get pods

In [ ]:
!kubectl delete pod products-6fd4b78888-8vv9c


In [ ]:
!kubectl delete deployment products
#!kubectl delete

In [ ]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

In [ ]:
!curl 34.70.215.215/api/products

In [ ]:
%cd ~/gcp-ms/backend/react-app
!cp ./.env.monolith ./.env
!sed -i 's+/service/orders+http://35.232.229.60/api/orders+g' ./.env
!sed -i 's+/service/products+http://35.232.229.60/api/products+g' ./.env

!cat ./.env

In [ ]:
!npm run build

In [ ]:
%cd ~/gcp-ms/backend/microservices/src/frontend
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0 .

## Internal Name space

# C : Cleanup Project

In [ ]:
!gcloud container clusters list

In [ ]:
!gcloud container clusters delete fancy-cluster


In [ ]:
!sed -i 's/unique-hull-new/{project_id}/g' ./RSA-Core/microservices/k8s-v2/frontend.yaml
!sed -i 's/nodeserver:v1.0.0/{server_app_tag}/g' ./RSA-Core/microservices/k8s-v2/frontend.yaml
!sed -i 's/nodeclient:v1.0.0/{client_app_tag}/g' ./RSA-Core/microservices/k8s-v2/frontend.yaml
!sed -i 's/47910159411-h5v9fv4pljka89t6l4b0td4c4sdnpolf.apps.googleusercontent.com/{oauth_id}/g' ./RSA-Core/microservices/k8s-v2/frontend.yaml
!sed -i 's/ghjebfjkrnfbgrvnknskf/{secret_key}/g' ./RSA-Core/microservices/k8s-v2/frontend.yaml